## Instructions ##
971 yeast

linear regression, snp weight vector, 选top 10%， 20%， 30%。。

分别用这些snp来再跑linear regression的model( no gene)

再把这些snp放到gene上，再跑一次linear regression model

 

神经网络input snp

最多两千个feature，用linear regression来筛选

 

还要考虑promoter， enhancer（先不搞）

 

1. 先选snp （Science paper：看snp preproccesing的procedure）<5 删掉

2. fit linear regression model

3. 选大约2000个

4. 再用神经网络 （pytorch）



In [158]:
import pandas as pd 
import numpy as np
import bisect as bs
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import scipy.stats as stat


In [159]:
SNP_url = "./1011geneSNP.csv"
Gene_url = "./GCF_000146045.2_R64_genomic.gff"

snp_yeast_matrix = pd.read_csv(SNP_url, sep = ',' , error_bad_lines=False)
del snp_yeast_matrix['#CHROM']
del snp_yeast_matrix['POS']
del snp_yeast_matrix['REF']
del snp_yeast_matrix['ALT']
del snp_yeast_matrix['ANN[*].GENE']
del snp_yeast_matrix['ANN[*].GENEID']
index_list = snp_yeast_matrix["ID"].tolist()
del snp_yeast_matrix['ID']
snp_yeast_matrix = snp_yeast_matrix.transpose()
column_names = list(snp_yeast_matrix.index)
to_delete = ["ABC", "ABF", "ASP", "BGS","BGF", "BHL","BHQ","BID","BIR","BKG","BKN","CFH","SACE_YAL","SACE_YBA",
             "SACE_YBB","SACE_YBM","SACE_YBN","SACE_YBO","SACE_YBP","SACE_YCJ","SACE_YCS","SACE_YCT","SACE_YCU","SACE_YCV","SACE_YCW",
             "SACE_YCX","SACE_YCY","SACE_YCZ","SACE_YDA","SACE_YDB","SACE_YDC","SACE_YDD", "SACE_YDE","SACE_YDF","SACE_YDG","SACE_YDH","SACE_YDI",
             "SACE_YDJ", "SACE_YDK", "SACE_YDL"]
print(len(to_delete))
index = 0
for index in range(len(to_delete)):
    curr = to_delete[index]
    curr = curr+"_"+curr
    to_delete[index] = curr
    index+=1
to_delete_index = []
for item in to_delete:
    curr_index = column_names.index(item)
    to_delete_index.append(curr_index)
snp_yeast_matrix = snp_yeast_matrix.drop(to_delete)
snp_yeast_matrix.head()

40


,0,1,2,3,4,5,6,7,8,9,...,18921,18922,18923,18924,18925,18926,18927,18928,18929,18930
AAA_AAA,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAB_AAB,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAC_AAC,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAD_AAD,0,0,0,0,1,1,1,0,0,0,...,0,0,1,1,1,1,0,0,1,1
AAE_AAE,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [160]:
pheno_url = "./phenoMatrix_35ConditionsNormalizedByYPD.csv"
pheno_yeast_matrix = pd.read_csv(pheno_url, delimiter='\t', error_bad_lines=False)
YPD6AU = pheno_yeast_matrix["YPD6AU"]
YPDBENOMYL200 = pheno_yeast_matrix["YPDBENOMYL200"]
YPDNACL1M = pheno_yeast_matrix["YPDNACL1M"]
YPDSDS = pheno_yeast_matrix["YPDSDS"]
YPGALACTOSE = pheno_yeast_matrix["YPGALACTOSE"]
YPGLYCEROL = pheno_yeast_matrix["YPGLYCEROL"]
YPRIBOSE = pheno_yeast_matrix["YPRIBOSE"]
YPSORBITOL = pheno_yeast_matrix["YPSORBITOL"]
total_run = 100

In [161]:
sum = 0
weight_vector_1 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPD6AU, test_size=0.2)
    lr1 = linear_model.LinearRegression()
    lr1.fit(x_train, y_train)
    temp = np.array(lr1.coef_)
    weight_vector_1 = [x + y for x, y in zip(weight_vector_1, temp)]
    predictions_1 = lr1.predict(x_test)
    pcc_1 = stat.pearsonr(predictions_1, y_test)
    counter+=1
    sum += pcc_1[0]

avg_assoc_1 = sum/total_run
avg_weight_vector_1 = np.divide(weight_vector_1, total_run)
print(avg_assoc_1)
print(avg_weight_vector_1)
top2000_index = (-avg_weight_vector_1).argsort()[:2000]
print (top2000_index)
print (avg_weight_vector_1[top2000_index])

0.013834694836189081
[-21637495.81609731 -43026792.60750684 -82235451.61591634 ...
   -364213.38674538   -675618.44586014   2274651.94929715]
[  10   39    8 ...  512 5550 4201]
[19394586.97699884 18875675.73484499 16879063.16618251 ...
  1349424.18783773  1348944.34342498  1348780.62660635]


In [162]:
sum = 0
weight_vector_3 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDNACL1M, test_size=0.2)
    lr3 = linear_model.LinearRegression()
    lr3.fit(x_train, y_train)
    temp = np.array(lr3.coef_)
    weight_vector_3 = [x + y for x, y in zip(weight_vector_3, temp)]
    predictions_3 = lr3.predict(x_test)
    pcc_3 = stat.pearsonr(predictions_3, y_test)
    counter+=1
    sum += pcc_3[0]

avg_assoc_3 = sum/total_run
avg_weight_vector_3 = np.divide(weight_vector_3, total_run)
print(avg_assoc_3)
print(avg_weight_vector_3)
top2000_index_3 = (-avg_weight_vector_3).argsort()[:2000]
print (top2000_index_3)
print (avg_weight_vector_3[top2000_index_3])

0.03166097752477239
[ 3.56082982e+07  5.94710464e+07 -5.35657508e+06 ... -1.73256580e+04
  4.95468120e+05  3.29731252e+05]
[    1     0     8 ... 13836  1294   445]
[59471046.44204026 35608298.21511064 24180145.6325774  ...
   747642.35556154   746824.74309788   746602.39799672]


In [163]:
sum = 0
weight_vector_4 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDSDS, test_size=0.2)
    lr4 = linear_model.LinearRegression()
    lr4.fit(x_train, y_train)
    temp = np.array(lr4.coef_)
    weight_vector_4 = [x + y for x, y in zip(weight_vector_4, temp)]
    predictions_4 = lr4.predict(x_test)
    pcc_4 = stat.pearsonr(predictions_4, y_test)
    counter+=1
    sum += pcc_4[0]

avg_assoc_4 = sum/total_run
avg_weight_vector_4 = np.divide(weight_vector_4, total_run)
print(avg_assoc_4)
print(avg_weight_vector_4)
top2000_index_4 = (-avg_weight_vector_4).argsort()[:2000]
print (top2000_index_4)
print (avg_weight_vector_4[top2000_index_4])

0.031298746062796916
[ 1.58519455e+07 -1.20134369e+08  7.82971636e+07 ...  1.04569976e+06
  1.22077029e+06  1.70616596e+06]
[   2   18   11 ... 1792 1795 1794]
[78297163.61122961 37697441.53566637 30157664.80763368 ...
  1266093.99725686  1266093.99725686  1266093.99725686]


In [164]:
sum = 0
weight_vector_2 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDBENOMYL200, test_size=0.2)
    lr2 = linear_model.LinearRegression()
    lr2.fit(x_train, y_train)
    temp = np.array(lr2.coef_)
    weight_vector_2 = [x + y for x, y in zip(weight_vector_2, temp)]
    predictions_2 = lr2.predict(x_test)
    pcc_2 = stat.pearsonr(predictions_2, y_test)
    counter+=1
    sum += pcc_2[0]

avg_assoc_2 = sum/total_run
avg_weight_vector_2 = np.divide(weight_vector_2, total_run)
print(avg_assoc_2)
print(avg_weight_vector_2)
top2000_index_2 = (-avg_weight_vector_2).argsort()[:2000]
print (top2000_index_2)
print (avg_weight_vector_2[top2000_index_2])

0.022228009830456738
[-39305564.07539475 -21681765.27085563  18845382.49125871 ...
     61867.24875883    938896.38677545     60925.37401646]
[    3     2    13 ... 15800 10805  5225]
[31172397.54388002 18845382.49125871 17966617.2309124  ...
   346533.36028601   346528.64860694   345959.6672411 ]


In [165]:
sum = 0
weight_vector_5 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGALACTOSE, test_size=0.2)
    lr5 = linear_model.LinearRegression()
    lr5.fit(x_train, y_train)
    temp = np.array(lr5.coef_)
    weight_vector_5 = [x + y for x, y in zip(weight_vector_5, temp)]
    predictions_5 = lr5.predict(x_test)
    pcc_5 = stat.pearsonr(predictions_5, y_test)
    counter+=1
    sum += pcc_5[0]

avg_assoc_5 = sum/total_run
avg_weight_vector_5 = np.divide(weight_vector_5, total_run)
print(avg_assoc_5)
print(avg_weight_vector_5)
top2000_index_5 = (-avg_weight_vector_5).argsort()[:2000]
print (top2000_index_5)
print (avg_weight_vector_5[top2000_index_5])

0.03959118132644832
[ 1.03370641e+08 -2.86514296e+08  1.11686418e+08 ... -2.18068322e+05
  2.04340616e+06 -1.14820363e+06]
[    3     2     0 ...  8303 12750  1504]
[1.38640519e+08 1.11686418e+08 1.03370641e+08 ... 2.24824565e+06
 2.24714087e+06 2.24679904e+06]


In [166]:
sum = 0
weight_vector_6 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGLYCEROL, test_size=0.2)
    lr6 = linear_model.LinearRegression()
    lr6.fit(x_train, y_train)
    temp = np.array(lr6.coef_)
    weight_vector_6 = [x + y for x, y in zip(weight_vector_6, temp)]
    predictions_6 = lr6.predict(x_test)
    pcc_6 = stat.pearsonr(predictions_6, y_test)
    counter+=1
    sum += pcc_6[0]

avg_assoc_6 = sum/total_run
avg_weight_vector_6 = np.divide(weight_vector_6, total_run)
print(avg_assoc_6)
print(avg_weight_vector_6)
top2000_index_6 = (-avg_weight_vector_6).argsort()[:2000]
print (top2000_index_6)
print (avg_weight_vector_6[top2000_index_6])

0.01496837620745466
[ 3.14329651e+08  1.67152312e+08  1.04431986e+08 ... -3.77306167e+05
  1.89699728e+06  2.95298913e+06]
[    0     1     2 ...  5581 17950  7807]
[3.14329651e+08 1.67152312e+08 1.04431986e+08 ... 1.59761210e+06
 1.59746238e+06 1.59616680e+06]


In [167]:
sum = 0
weight_vector_7 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPRIBOSE, test_size=0.2)
    lr7 = linear_model.LinearRegression()
    lr7.fit(x_train, y_train)
    temp = np.array(lr7.coef_)
    weight_vector_7 = [x + y for x, y in zip(weight_vector_7, temp)]
    predictions_7 = lr7.predict(x_test)
    pcc_7 = stat.pearsonr(predictions_7, y_test)
    counter+=1
    sum += pcc_7[0]

avg_assoc_7 = sum/total_run
avg_weight_vector_7 = np.divide(weight_vector_7, total_run)
print(avg_assoc_7)
print(avg_weight_vector_7)
top2000_index_7 = (-avg_weight_vector_7).argsort()[:2000]
print (top2000_index_7)
print (avg_weight_vector_7[top2000_index_7])

0.03421475631705529
[-22967749.63888364  36314791.28746554  33444986.85166423 ...
   -130132.85717752   -940427.2342694   -1102046.22017298]
[    5     1     2 ... 10926 14397 15613]
[40457936.78645854 36314791.28746554 33444986.85166423 ...
  1252638.76997886  1251937.31276146  1251843.47016033]


In [168]:
sum = 0
weight_vector_8 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPSORBITOL, test_size=0.2)
    lr8 = linear_model.LinearRegression()
    lr8.fit(x_train, y_train)
    temp = np.array(lr8.coef_)
    weight_vector_8 = [x + y for x, y in zip(weight_vector_8, temp)]
    predictions_8 = lr8.predict(x_test)
    pcc_8 = stat.pearsonr(predictions_8, y_test)
    counter+=1
    sum += pcc_8[0]

avg_assoc_8 = sum/total_run
avg_weight_vector_8 = np.divide(weight_vector_8, total_run)
print(avg_assoc_8)
print(avg_weight_vector_8)
top2000_index_8 = (-avg_weight_vector_8).argsort()[:2000]
print (top2000_index_8)
print (avg_weight_vector_8[top2000_index_8])

0.040003485263229395
[-1.13793757e+08 -2.68497384e+07 -2.86100601e+07 ... -3.54971981e+05
  9.54428813e+05 -2.04602446e+06]
[   3   16   11 ...  351 6091 2290]
[52808899.41873073 23605133.50449315 22472132.79453375 ...
   937155.95972348   937116.1697107    937090.33159536]


In [169]:
df_columns = ["YPD6AU", "YPDBENOMYL200", "YPDNACL1M", "YPDSDS", "YPGALACTOSE", "YPGLYCEROL", "YPRIBOSE", "YPSORBITOL"]
index_df = pd.DataFrame(list(zip(top2000_index, top2000_index_2, top2000_index_3, top2000_index_4, top2000_index_5, top2000_index_6, top2000_index_7, top2000_index_8)), columns = df_columns)


最后一层hiddeen nueron 数量为一

2000 1000 500 100 1 

adam optimize 要

batch normalization 

.cuda 挪到gpu

batch training 

batch

# Neural Network Class #

In [170]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as dt

class Net(nn.Module):
    def __init__(self, input_size, h1, h2, h3, output_size, lr):
        super(Net, self).__init__()
        self.l1 = nn.Linear(input_size, h1)
        self.l2 = nn.Linear(h1, h2)
        self.l3 = nn.Linear(h2, h3)
        self.l4 = nn.Linear(h3, output_size)
        self.relu = nn.ReLU()
        
        self.loss_fn = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr)

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        return out
    
    def train_net(self, num_epochs, train_loader, test_loader):
        max_rsq = 0
        max_epoch = 0
        for epoch in range (num_epochs):
            total_loss = 0
            counter = 0
            for i, (data, target) in enumerate(train_loader):
                self.zero_grad()
                data = data.to(device)
                target = target.to(device)
                output = self.forward(data).to(device)
                loss = self.loss_fn(output, target)
                total_loss += loss
                counter += 1
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            
            # empty arrays for empty tensor 
            empty1 = np.empty((0,1))
            empty2 = np.empty((0,1))
            output_tensor = torch.tensor(empty1, dtype = torch.float)
            target_tensor = torch.tensor(empty2, dtype = torch.float)
            # concatenate output and target to the empty tensors 
            for i, (data, target) in enumerate(test_loader): 
                data = data.to(device) 
                target = target.to(device) 
                output = self.forward(data).to(device)
                target = torch.reshape(target, (len(target),1))
                output_tensor = torch.cat((output_tensor, output.cpu()))
                target_tensor = torch.cat((target_tensor, target.cpu()))
            
            #convert the tensor into numpy arrays and calculate correlation 
            output_array = output_tensor.detach().numpy()
            target_array = target_tensor.detach().numpy()
            if ((output_array.shape[0]!=target_array.shape[0]) or (output_array.shape[1]!=target_array.shape[1])):
                raise Exception('Dimension mismatch')
            result = stat.pearsonr(target_array, output_array)
            print("Correlation: %f \n P-Value: %f" %(result[0], result[1])) 
            if(math.fabs(result[0])>math.fabs(max_rsq)):
                max_rsq = result[0]
                max_epoch = epoch
            epoch += 1
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                .format(epoch, num_epochs, total_loss/len(train_loader)))
    
        print("Max correlation is: {} at {}th epoch".format(max_rsq, max_epoch))

## Net Config Section ##

In [227]:
device = torch.device('cuda')
input_size = 2000
h1 = 1000
h2 = 200
h3 = 100
output_size = 1
lr = 0.0001
num_epochs = 400
batch_size = 200
split = 0.8


In [228]:
traning_data = torch.tensor(snp_yeast_matrix[top2000_index].values, dtype = torch.float)
answer = torch.tensor(YPD6AU, dtype = torch.float)
train_set = dt.TensorDataset(traning_data, answer)
train_size = int(split * len(train_set))
test_size = len(train_set) - train_size 

train_dataset, test_dataset = dt.random_split(train_set, [train_size, test_size])
train_loader = dt.DataLoader(dataset=train_dataset, batch_size=batch_size)

net = Net(input_size, h1, h2, h3, output_size, lr).to(device)

test_loader = dt.DataLoader(dataset=test_dataset, batch_size=batch_size)

net.train_net(num_epochs, train_loader, test_loader)

Correlation: -0.131267 
 P-Value: 0.067373
Epoch [1/400], Loss: 0.0445
Correlation: -0.043274 
 P-Value: 0.548054
Epoch [2/400], Loss: 0.0234
Correlation: 0.173965 
 P-Value: 0.015006
Epoch [3/400], Loss: 0.0200
Correlation: 0.119297 
 P-Value: 0.096689
Epoch [4/400], Loss: 0.0201
Correlation: -0.109951 
 P-Value: 0.125977
Epoch [5/400], Loss: 0.0192
Correlation: -0.221252 
 P-Value: 0.001881
Epoch [6/400], Loss: 0.0190
Correlation: -0.278400 
 P-Value: 0.000081
Epoch [7/400], Loss: 0.0188
Correlation: -0.262900 
 P-Value: 0.000205
Epoch [8/400], Loss: 0.0187
Correlation: -0.131456 
 P-Value: 0.066976
Epoch [9/400], Loss: 0.0187
Correlation: -0.160005 
 P-Value: 0.025457
Epoch [10/400], Loss: 0.0186
Correlation: -0.105167 
 P-Value: 0.143413
Epoch [11/400], Loss: 0.0186
Correlation: -0.076958 
 P-Value: 0.284922
Epoch [12/400], Loss: 0.0185
Correlation: -0.050497 
 P-Value: 0.483262
Epoch [13/400], Loss: 0.0185
Correlation: -0.035468 
 P-Value: 0.622543
Epoch [14/400], Loss: 0.0185
Cor

Epoch [114/400], Loss: 0.0183
Correlation: -0.048714 
 P-Value: 0.498868
Epoch [115/400], Loss: 0.0183
Correlation: -0.048443 
 P-Value: 0.501258
Epoch [116/400], Loss: 0.0183
Correlation: -0.048843 
 P-Value: 0.497721
Epoch [117/400], Loss: 0.0183
Correlation: -0.048706 
 P-Value: 0.498933
Epoch [118/400], Loss: 0.0183
Correlation: -0.050220 
 P-Value: 0.485668
Epoch [119/400], Loss: 0.0183
Correlation: -0.049635 
 P-Value: 0.490765
Epoch [120/400], Loss: 0.0183
Correlation: -0.046675 
 P-Value: 0.517016
Epoch [121/400], Loss: 0.0183
Correlation: -0.047419 
 P-Value: 0.510354
Epoch [122/400], Loss: 0.0183
Correlation: -0.049606 
 P-Value: 0.491022
Epoch [123/400], Loss: 0.0183
Correlation: -0.049483 
 P-Value: 0.492097
Epoch [124/400], Loss: 0.0183
Correlation: -0.048266 
 P-Value: 0.502830
Epoch [125/400], Loss: 0.0183
Correlation: -0.047495 
 P-Value: 0.509681
Epoch [126/400], Loss: 0.0183
Correlation: -0.047518 
 P-Value: 0.509474
Epoch [127/400], Loss: 0.0183
Correlation: -0.04853

Epoch [226/400], Loss: 0.0183
Correlation: -0.049831 
 P-Value: 0.489058
Epoch [227/400], Loss: 0.0183
Correlation: -0.052924 
 P-Value: 0.462458
Epoch [228/400], Loss: 0.0183
Correlation: -0.055075 
 P-Value: 0.444443
Epoch [229/400], Loss: 0.0183
Correlation: -0.054164 
 P-Value: 0.452017
Epoch [230/400], Loss: 0.0183
Correlation: -0.051517 
 P-Value: 0.474451
Epoch [231/400], Loss: 0.0183
Correlation: -0.048910 
 P-Value: 0.497135
Epoch [232/400], Loss: 0.0183
Correlation: -0.048777 
 P-Value: 0.498307
Epoch [233/400], Loss: 0.0183
Correlation: -0.051605 
 P-Value: 0.473702
Epoch [234/400], Loss: 0.0183
Correlation: -0.053787 
 P-Value: 0.455177
Epoch [235/400], Loss: 0.0183
Correlation: -0.053344 
 P-Value: 0.458907
Epoch [236/400], Loss: 0.0183
Correlation: -0.050796 
 P-Value: 0.480672
Epoch [237/400], Loss: 0.0183
Correlation: -0.049099 
 P-Value: 0.495476
Epoch [238/400], Loss: 0.0183
Correlation: -0.049313 
 P-Value: 0.493583
Epoch [239/400], Loss: 0.0183
Correlation: -0.05078

Epoch [338/400], Loss: 0.0183
Correlation: -0.057315 
 P-Value: 0.426109
Epoch [339/400], Loss: 0.0183
Correlation: -0.057716 
 P-Value: 0.422870
Epoch [340/400], Loss: 0.0183
Correlation: -0.057709 
 P-Value: 0.422932
Epoch [341/400], Loss: 0.0183
Correlation: -0.057365 
 P-Value: 0.425698
Epoch [342/400], Loss: 0.0183
Correlation: -0.056924 
 P-Value: 0.429280
Epoch [343/400], Loss: 0.0183
Correlation: -0.056721 
 P-Value: 0.430926
Epoch [344/400], Loss: 0.0183
Correlation: -0.057120 
 P-Value: 0.427694
Epoch [345/400], Loss: 0.0183
Correlation: -0.057401 
 P-Value: 0.425417
Epoch [346/400], Loss: 0.0183
Correlation: -0.057529 
 P-Value: 0.424383
Epoch [347/400], Loss: 0.0183
Correlation: -0.057814 
 P-Value: 0.422081
Epoch [348/400], Loss: 0.0183
Correlation: -0.057339 
 P-Value: 0.425916
Epoch [349/400], Loss: 0.0183
Correlation: -0.056776 
 P-Value: 0.430481
Epoch [350/400], Loss: 0.0183
Correlation: -0.056988 
 P-Value: 0.428753
Epoch [351/400], Loss: 0.0183
Correlation: -0.05677

In [229]:
traning_data_2 = torch.tensor(snp_yeast_matrix[top2000_index_2].values, dtype = torch.float)
answer_2 = torch.tensor(YPDBENOMYL200, dtype = torch.float)
train_set_2 = dt.TensorDataset(traning_data_2, answer_2)

train_dataset_2, test_dataset_2 = dt.random_split(train_set_2, [train_size, test_size])
train_loader_2 = dt.DataLoader(dataset=train_dataset_2, batch_size=batch_size)
test_loader_2 = dt.DataLoader(dataset=test_dataset_2, batch_size=batch_size)

net_2 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_2.train_net(num_epochs, train_loader_2, test_loader_2)

Correlation: 0.168656 
 P-Value: 0.018427
Epoch [1/400], Loss: 0.0423
Correlation: 0.150619 
 P-Value: 0.035575
Epoch [2/400], Loss: 0.0172
Correlation: -0.003475 
 P-Value: 0.961507
Epoch [3/400], Loss: 0.0125
Correlation: 0.063982 
 P-Value: 0.374203
Epoch [4/400], Loss: 0.0124
Correlation: 0.177919 
 P-Value: 0.012831
Epoch [5/400], Loss: 0.0107
Correlation: 0.145327 
 P-Value: 0.042651
Epoch [6/400], Loss: 0.0105
Correlation: 0.114856 
 P-Value: 0.109853
Epoch [7/400], Loss: 0.0103
Correlation: 0.137615 
 P-Value: 0.055051
Epoch [8/400], Loss: 0.0101
Correlation: 0.101803 
 P-Value: 0.156733
Epoch [9/400], Loss: 0.0100
Correlation: 0.170628 
 P-Value: 0.017085
Epoch [10/400], Loss: 0.0099
Correlation: 0.171018 
 P-Value: 0.016829
Epoch [11/400], Loss: 0.0099
Correlation: 0.129130 
 P-Value: 0.071995
Epoch [12/400], Loss: 0.0098
Correlation: 0.143838 
 P-Value: 0.044844
Epoch [13/400], Loss: 0.0098
Correlation: 0.118405 
 P-Value: 0.099226
Epoch [14/400], Loss: 0.0097
Correlation: 0

 P-Value: 0.407411
Epoch [116/400], Loss: 0.0095
Correlation: 0.059530 
 P-Value: 0.408420
Epoch [117/400], Loss: 0.0095
Correlation: 0.059425 
 P-Value: 0.409243
Epoch [118/400], Loss: 0.0095
Correlation: 0.059775 
 P-Value: 0.406488
Epoch [119/400], Loss: 0.0095
Correlation: 0.059990 
 P-Value: 0.404803
Epoch [120/400], Loss: 0.0095
Correlation: 0.059909 
 P-Value: 0.405435
Epoch [121/400], Loss: 0.0095
Correlation: 0.059691 
 P-Value: 0.407148
Epoch [122/400], Loss: 0.0095
Correlation: 0.059613 
 P-Value: 0.407765
Epoch [123/400], Loss: 0.0095
Correlation: 0.059534 
 P-Value: 0.408389
Epoch [124/400], Loss: 0.0095
Correlation: 0.059541 
 P-Value: 0.408337
Epoch [125/400], Loss: 0.0095
Correlation: 0.059789 
 P-Value: 0.406374
Epoch [126/400], Loss: 0.0095
Correlation: 0.059868 
 P-Value: 0.405756
Epoch [127/400], Loss: 0.0095
Correlation: 0.059839 
 P-Value: 0.405987
Epoch [128/400], Loss: 0.0095
Correlation: 0.059633 
 P-Value: 0.407604
Epoch [129/400], Loss: 0.0095
Correlation: 0.

 P-Value: 0.226147
Epoch [230/400], Loss: 0.0095
Correlation: 0.072996 
 P-Value: 0.310519
Epoch [231/400], Loss: 0.0095
Correlation: 0.047100 
 P-Value: 0.513207
Epoch [232/400], Loss: 0.0095
Correlation: 0.040905 
 P-Value: 0.570189
Epoch [233/400], Loss: 0.0095
Correlation: 0.054727 
 P-Value: 0.447330
Epoch [234/400], Loss: 0.0095
Correlation: 0.069376 
 P-Value: 0.335186
Epoch [235/400], Loss: 0.0095
Correlation: 0.068910 
 P-Value: 0.338453
Epoch [236/400], Loss: 0.0095
Correlation: 0.058855 
 P-Value: 0.413758
Epoch [237/400], Loss: 0.0095
Correlation: 0.052846 
 P-Value: 0.463112
Epoch [238/400], Loss: 0.0095
Correlation: 0.056505 
 P-Value: 0.432688
Epoch [239/400], Loss: 0.0095
Correlation: 0.063654 
 P-Value: 0.376657
Epoch [240/400], Loss: 0.0095
Correlation: 0.065213 
 P-Value: 0.365063
Epoch [241/400], Loss: 0.0095
Correlation: 0.060613 
 P-Value: 0.399931
Epoch [242/400], Loss: 0.0095
Correlation: 0.056240 
 P-Value: 0.434852
Epoch [243/400], Loss: 0.0095
Correlation: 0.

 P-Value: 0.373744
Epoch [344/400], Loss: 0.0095
Correlation: 0.064160 
 P-Value: 0.372872
Epoch [345/400], Loss: 0.0095
Correlation: 0.064543 
 P-Value: 0.370023
Epoch [346/400], Loss: 0.0095
Correlation: 0.064831 
 P-Value: 0.367883
Epoch [347/400], Loss: 0.0095
Correlation: 0.064606 
 P-Value: 0.369549
Epoch [348/400], Loss: 0.0095
Correlation: 0.064255 
 P-Value: 0.372169
Epoch [349/400], Loss: 0.0095
Correlation: 0.064167 
 P-Value: 0.372823
Epoch [350/400], Loss: 0.0095
Correlation: 0.064329 
 P-Value: 0.371612
Epoch [351/400], Loss: 0.0095
Correlation: 0.064591 
 P-Value: 0.369666
Epoch [352/400], Loss: 0.0095
Correlation: 0.064726 
 P-Value: 0.368664
Epoch [353/400], Loss: 0.0095
Correlation: 0.064683 
 P-Value: 0.368982
Epoch [354/400], Loss: 0.0095
Correlation: 0.064589 
 P-Value: 0.369683
Epoch [355/400], Loss: 0.0095
Correlation: 0.064481 
 P-Value: 0.370479
Epoch [356/400], Loss: 0.0095
Correlation: 0.064500 
 P-Value: 0.370338
Epoch [357/400], Loss: 0.0095
Correlation: 0.

In [230]:
traning_data_3 = torch.tensor(snp_yeast_matrix[top2000_index_3].values, dtype = torch.float)
answer_3 = torch.tensor(YPDNACL1M, dtype = torch.float)
train_set_3 = dt.TensorDataset(traning_data_3, answer_3)

train_dataset_3, test_dataset_3 = dt.random_split(train_set_3, [train_size, test_size])
train_loader_3 = dt.DataLoader(dataset=train_dataset_3, batch_size=batch_size)
test_loader_3 = dt.DataLoader(dataset=test_dataset_3, batch_size=batch_size)

net_3 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_3.train_net(num_epochs, train_loader_3, test_loader_3)

Correlation: 0.018280 
 P-Value: 0.799773
Epoch [1/400], Loss: 0.0232
Correlation: 0.129003 
 P-Value: 0.072278
Epoch [2/400], Loss: 0.0121
Correlation: 0.091186 
 P-Value: 0.204871
Epoch [3/400], Loss: 0.0117
Correlation: -0.049004 
 P-Value: 0.496305
Epoch [4/400], Loss: 0.0108
Correlation: -0.079926 
 P-Value: 0.266693
Epoch [5/400], Loss: 0.0107
Correlation: -0.041621 
 P-Value: 0.563456
Epoch [6/400], Loss: 0.0106
Correlation: -0.014867 
 P-Value: 0.836575
Epoch [7/400], Loss: 0.0105
Correlation: -0.045401 
 P-Value: 0.528544
Epoch [8/400], Loss: 0.0104
Correlation: -0.032431 
 P-Value: 0.652659
Epoch [9/400], Loss: 0.0104
Correlation: -0.020659 
 P-Value: 0.774361
Epoch [10/400], Loss: 0.0104
Correlation: -0.037459 
 P-Value: 0.603132
Epoch [11/400], Loss: 0.0104
Correlation: -0.052346 
 P-Value: 0.467368
Epoch [12/400], Loss: 0.0103
Correlation: -0.032753 
 P-Value: 0.649431
Epoch [13/400], Loss: 0.0103
Correlation: -0.000388 
 P-Value: 0.995319
Epoch [14/400], Loss: 0.0103
Corr

Epoch [114/400], Loss: 0.0102
Correlation: -0.004905 
 P-Value: 0.945785
Epoch [115/400], Loss: 0.0102
Correlation: -0.005197 
 P-Value: 0.942463
Epoch [116/400], Loss: 0.0102
Correlation: -0.005384 
 P-Value: 0.940412
Epoch [117/400], Loss: 0.0102
Correlation: -0.006641 
 P-Value: 0.926582
Epoch [118/400], Loss: 0.0102
Correlation: -0.007115 
 P-Value: 0.921377
Epoch [119/400], Loss: 0.0102
Correlation: -0.004536 
 P-Value: 0.949833
Epoch [120/400], Loss: 0.0102
Correlation: -0.006493 
 P-Value: 0.928233
Epoch [121/400], Loss: 0.0102
Correlation: -0.006591 
 P-Value: 0.927078
Epoch [122/400], Loss: 0.0102
Correlation: -0.004777 
 P-Value: 0.947146
Epoch [123/400], Loss: 0.0102
Correlation: -0.006536 
 P-Value: 0.927779
Epoch [124/400], Loss: 0.0102
Correlation: -0.005759 
 P-Value: 0.936282
Epoch [125/400], Loss: 0.0102
Correlation: -0.004748 
 P-Value: 0.947491
Epoch [126/400], Loss: 0.0102
Correlation: -0.006337 
 P-Value: 0.929924
Epoch [127/400], Loss: 0.0102
Correlation: -0.00603

Epoch [226/400], Loss: 0.0102
Correlation: -0.011793 
 P-Value: 0.870033
Epoch [227/400], Loss: 0.0102
Correlation: -0.011512 
 P-Value: 0.873107
Epoch [228/400], Loss: 0.0102
Correlation: -0.012332 
 P-Value: 0.864126
Epoch [229/400], Loss: 0.0102
Correlation: -0.010421 
 P-Value: 0.885034
Epoch [230/400], Loss: 0.0102
Correlation: -0.009160 
 P-Value: 0.898892
Epoch [231/400], Loss: 0.0102
Correlation: -0.009369 
 P-Value: 0.896596
Epoch [232/400], Loss: 0.0102
Correlation: -0.007797 
 P-Value: 0.913850
Epoch [233/400], Loss: 0.0102
Correlation: -0.010714 
 P-Value: 0.881821
Epoch [234/400], Loss: 0.0102
Correlation: -0.010983 
 P-Value: 0.878874
Epoch [235/400], Loss: 0.0102
Correlation: -0.012893 
 P-Value: 0.858020
Epoch [236/400], Loss: 0.0102
Correlation: -0.010163 
 P-Value: 0.887858
Epoch [237/400], Loss: 0.0102
Correlation: -0.009541 
 P-Value: 0.894693
Epoch [238/400], Loss: 0.0102
Correlation: -0.011375 
 P-Value: 0.874588
Epoch [239/400], Loss: 0.0102
Correlation: -0.00908

Epoch [338/400], Loss: 0.0102
Correlation: -0.016284 
 P-Value: 0.821239
Epoch [339/400], Loss: 0.0102
Correlation: -0.015547 
 P-Value: 0.829211
Epoch [340/400], Loss: 0.0102
Correlation: -0.014235 
 P-Value: 0.843441
Epoch [341/400], Loss: 0.0102
Correlation: -0.013876 
 P-Value: 0.847311
Epoch [342/400], Loss: 0.0102
Correlation: -0.015320 
 P-Value: 0.831677
Epoch [343/400], Loss: 0.0102
Correlation: -0.017959 
 P-Value: 0.803214
Epoch [344/400], Loss: 0.0102
Correlation: -0.017807 
 P-Value: 0.804842
Epoch [345/400], Loss: 0.0102
Correlation: -0.015937 
 P-Value: 0.824994
Epoch [346/400], Loss: 0.0102
Correlation: -0.015027 
 P-Value: 0.834825
Epoch [347/400], Loss: 0.0102
Correlation: -0.016268 
 P-Value: 0.821417
Epoch [348/400], Loss: 0.0102
Correlation: -0.019200 
 P-Value: 0.789916
Epoch [349/400], Loss: 0.0102
Correlation: -0.022436 
 P-Value: 0.755555
Epoch [350/400], Loss: 0.0102
Correlation: -0.018878 
 P-Value: 0.793364
Epoch [351/400], Loss: 0.0102
Correlation: -0.01633

In [231]:
traning_data_4 = torch.tensor(snp_yeast_matrix[top2000_index_4].values, dtype = torch.float)
answer_4 = torch.tensor(YPDSDS, dtype = torch.float)
train_set_4 = dt.TensorDataset(traning_data_4, answer_4)

train_dataset_4, test_dataset_4 = dt.random_split(train_set_4, [train_size, test_size])
train_loader_4 = dt.DataLoader(dataset=train_dataset_4, batch_size=batch_size)
test_loader_4 = dt.DataLoader(dataset=test_dataset_4, batch_size=batch_size)

net_4 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_4.train_net(num_epochs, train_loader_4, test_loader_4)

Correlation: 0.108798 
 P-Value: 0.130022
Epoch [1/400], Loss: 0.0637
Correlation: -0.021251 
 P-Value: 0.768081
Epoch [2/400], Loss: 0.0347
Correlation: -0.092000 
 P-Value: 0.200840
Epoch [3/400], Loss: 0.0312
Correlation: -0.015883 
 P-Value: 0.825561
Epoch [4/400], Loss: 0.0296
Correlation: -0.055437 
 P-Value: 0.441451
Epoch [5/400], Loss: 0.0284
Correlation: -0.079431 
 P-Value: 0.269675
Epoch [6/400], Loss: 0.0280
Correlation: -0.105236 
 P-Value: 0.143150
Epoch [7/400], Loss: 0.0278
Correlation: -0.151104 
 P-Value: 0.034979
Epoch [8/400], Loss: 0.0276
Correlation: -0.084280 
 P-Value: 0.241429
Epoch [9/400], Loss: 0.0275
Correlation: -0.076835 
 P-Value: 0.285691
Epoch [10/400], Loss: 0.0275
Correlation: -0.096172 
 P-Value: 0.181081
Epoch [11/400], Loss: 0.0274
Correlation: -0.092221 
 P-Value: 0.199757
Epoch [12/400], Loss: 0.0273
Correlation: -0.103100 
 P-Value: 0.151492
Epoch [13/400], Loss: 0.0273
Correlation: -0.067667 
 P-Value: 0.347255
Epoch [14/400], Loss: 0.0272
Co

Epoch [114/400], Loss: 0.0270
Correlation: -0.011516 
 P-Value: 0.873051
Epoch [115/400], Loss: 0.0270
Correlation: -0.011073 
 P-Value: 0.877899
Epoch [116/400], Loss: 0.0270
Correlation: -0.012009 
 P-Value: 0.867680
Epoch [117/400], Loss: 0.0270
Correlation: -0.011865 
 P-Value: 0.869235
Epoch [118/400], Loss: 0.0270
Correlation: -0.011434 
 P-Value: 0.873931
Epoch [119/400], Loss: 0.0270
Correlation: -0.012203 
 P-Value: 0.865530
Epoch [120/400], Loss: 0.0270
Correlation: -0.012486 
 P-Value: 0.862475
Epoch [121/400], Loss: 0.0270
Correlation: -0.011652 
 P-Value: 0.871560
Epoch [122/400], Loss: 0.0270
Correlation: -0.011716 
 P-Value: 0.870836
Epoch [123/400], Loss: 0.0270
Correlation: -0.012359 
 P-Value: 0.863862
Epoch [124/400], Loss: 0.0270
Correlation: -0.012103 
 P-Value: 0.866628
Epoch [125/400], Loss: 0.0270
Correlation: -0.012736 
 P-Value: 0.859744
Epoch [126/400], Loss: 0.0270
Correlation: -0.013517 
 P-Value: 0.851238
Epoch [127/400], Loss: 0.0270
Correlation: -0.01360

Epoch [226/400], Loss: 0.0270
Correlation: -0.014447 
 P-Value: 0.841118
Epoch [227/400], Loss: 0.0270
Correlation: -0.013972 
 P-Value: 0.846264
Epoch [228/400], Loss: 0.0270
Correlation: -0.014396 
 P-Value: 0.841678
Epoch [229/400], Loss: 0.0270
Correlation: -0.014158 
 P-Value: 0.844262
Epoch [230/400], Loss: 0.0270
Correlation: -0.013761 
 P-Value: 0.848600
Epoch [231/400], Loss: 0.0270
Correlation: -0.013928 
 P-Value: 0.846775
Epoch [232/400], Loss: 0.0270
Correlation: -0.013895 
 P-Value: 0.847101
Epoch [233/400], Loss: 0.0270
Correlation: -0.013922 
 P-Value: 0.846821
Epoch [234/400], Loss: 0.0270
Correlation: -0.014075 
 P-Value: 0.845179
Epoch [235/400], Loss: 0.0270
Correlation: -0.013907 
 P-Value: 0.846984
Epoch [236/400], Loss: 0.0270
Correlation: -0.013410 
 P-Value: 0.852394
Epoch [237/400], Loss: 0.0270
Correlation: -0.013360 
 P-Value: 0.852927
Epoch [238/400], Loss: 0.0270
Correlation: -0.014130 
 P-Value: 0.844559
Epoch [239/400], Loss: 0.0270
Correlation: -0.01420

Epoch [338/400], Loss: 0.0270
Correlation: -0.025990 
 P-Value: 0.718351
Epoch [339/400], Loss: 0.0270
Correlation: -0.025929 
 P-Value: 0.718983
Epoch [340/400], Loss: 0.0270
Correlation: -0.025260 
 P-Value: 0.725945
Epoch [341/400], Loss: 0.0270
Correlation: -0.024324 
 P-Value: 0.735720
Epoch [342/400], Loss: 0.0270
Correlation: -0.024140 
 P-Value: 0.737638
Epoch [343/400], Loss: 0.0270
Correlation: -0.024365 
 P-Value: 0.735285
Epoch [344/400], Loss: 0.0270
Correlation: -0.025779 
 P-Value: 0.720551
Epoch [345/400], Loss: 0.0270
Correlation: -0.026065 
 P-Value: 0.717579
Epoch [346/400], Loss: 0.0270
Correlation: -0.025793 
 P-Value: 0.720407
Epoch [347/400], Loss: 0.0270
Correlation: -0.024984 
 P-Value: 0.728826
Epoch [348/400], Loss: 0.0270
Correlation: -0.023890 
 P-Value: 0.740265
Epoch [349/400], Loss: 0.0270
Correlation: -0.024085 
 P-Value: 0.738221
Epoch [350/400], Loss: 0.0270
Correlation: -0.025003 
 P-Value: 0.728627
Epoch [351/400], Loss: 0.0270
Correlation: -0.02666

In [232]:
traning_data_5 = torch.tensor(snp_yeast_matrix[top2000_index_5].values, dtype = torch.float)
answer_5 = torch.tensor(YPGALACTOSE, dtype = torch.float)
train_set_5 = dt.TensorDataset(traning_data_5, answer_5)

train_dataset_5, test_dataset_5 = dt.random_split(train_set_5, [train_size, test_size])
train_loader_5 = dt.DataLoader(dataset=train_dataset_5, batch_size=batch_size)
test_loader_5 = dt.DataLoader(dataset=test_dataset_5, batch_size=batch_size)

net_5 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_5.train_net(num_epochs, train_loader_5, test_loader_5)

Correlation: 0.199024 
 P-Value: 0.005282
Epoch [1/400], Loss: 0.7422
Correlation: 0.197167 
 P-Value: 0.005731
Epoch [2/400], Loss: 0.4165
Correlation: 0.185499 
 P-Value: 0.009424
Epoch [3/400], Loss: 0.2826
Correlation: 0.101632 
 P-Value: 0.157433
Epoch [4/400], Loss: 0.2113
Correlation: -0.056533 
 P-Value: 0.432456
Epoch [5/400], Loss: 0.1535
Correlation: -0.090202 
 P-Value: 0.209818
Epoch [6/400], Loss: 0.1555
Correlation: -0.032183 
 P-Value: 0.655136
Epoch [7/400], Loss: 0.1462
Correlation: 0.026434 
 P-Value: 0.713754
Epoch [8/400], Loss: 0.1322
Correlation: 0.030958 
 P-Value: 0.667468
Epoch [9/400], Loss: 0.1242
Correlation: 0.031874 
 P-Value: 0.658246
Epoch [10/400], Loss: 0.1185
Correlation: 0.007977 
 P-Value: 0.911854
Epoch [11/400], Loss: 0.1123
Correlation: -0.069425 
 P-Value: 0.334847
Epoch [12/400], Loss: 0.1088
Correlation: -0.072830 
 P-Value: 0.311623
Epoch [13/400], Loss: 0.1068
Correlation: -0.015318 
 P-Value: 0.831698
Epoch [14/400], Loss: 0.1048
Correlati

Epoch [114/400], Loss: 0.0944
Correlation: -0.037853 
 P-Value: 0.599322
Epoch [115/400], Loss: 0.0944
Correlation: -0.038006 
 P-Value: 0.597851
Epoch [116/400], Loss: 0.0944
Correlation: -0.038016 
 P-Value: 0.597753
Epoch [117/400], Loss: 0.0944
Correlation: -0.038167 
 P-Value: 0.596290
Epoch [118/400], Loss: 0.0944
Correlation: -0.038297 
 P-Value: 0.595045
Epoch [119/400], Loss: 0.0944
Correlation: -0.038413 
 P-Value: 0.593924
Epoch [120/400], Loss: 0.0944
Correlation: -0.038387 
 P-Value: 0.594170
Epoch [121/400], Loss: 0.0944
Correlation: -0.038615 
 P-Value: 0.591991
Epoch [122/400], Loss: 0.0944
Correlation: -0.038646 
 P-Value: 0.591687
Epoch [123/400], Loss: 0.0944
Correlation: -0.038824 
 P-Value: 0.589975
Epoch [124/400], Loss: 0.0944
Correlation: -0.038870 
 P-Value: 0.589534
Epoch [125/400], Loss: 0.0944
Correlation: -0.038945 
 P-Value: 0.588822
Epoch [126/400], Loss: 0.0944
Correlation: -0.039018 
 P-Value: 0.588120
Epoch [127/400], Loss: 0.0944
Correlation: -0.03905

Epoch [226/400], Loss: 0.0944
Correlation: -0.041012 
 P-Value: 0.569180
Epoch [227/400], Loss: 0.0944
Correlation: -0.040955 
 P-Value: 0.569715
Epoch [228/400], Loss: 0.0944
Correlation: -0.040995 
 P-Value: 0.569338
Epoch [229/400], Loss: 0.0944
Correlation: -0.040905 
 P-Value: 0.570189
Epoch [230/400], Loss: 0.0944
Correlation: -0.040903 
 P-Value: 0.570210
Epoch [231/400], Loss: 0.0944
Correlation: -0.040853 
 P-Value: 0.570684
Epoch [232/400], Loss: 0.0944
Correlation: -0.040922 
 P-Value: 0.570024
Epoch [233/400], Loss: 0.0944
Correlation: -0.040982 
 P-Value: 0.569461
Epoch [234/400], Loss: 0.0944
Correlation: -0.040989 
 P-Value: 0.569393
Epoch [235/400], Loss: 0.0944
Correlation: -0.040903 
 P-Value: 0.570210
Epoch [236/400], Loss: 0.0944
Correlation: -0.040915 
 P-Value: 0.570093
Epoch [237/400], Loss: 0.0944
Correlation: -0.040849 
 P-Value: 0.570719
Epoch [238/400], Loss: 0.0944
Correlation: -0.040872 
 P-Value: 0.570505
Epoch [239/400], Loss: 0.0944
Correlation: -0.04097

Epoch [338/400], Loss: 0.0944
Correlation: -0.042883 
 P-Value: 0.551671
Epoch [339/400], Loss: 0.0944
Correlation: -0.043251 
 P-Value: 0.548265
Epoch [340/400], Loss: 0.0944
Correlation: -0.043129 
 P-Value: 0.549396
Epoch [341/400], Loss: 0.0944
Correlation: -0.042752 
 P-Value: 0.552887
Epoch [342/400], Loss: 0.0944
Correlation: -0.042295 
 P-Value: 0.557149
Epoch [343/400], Loss: 0.0944
Correlation: -0.042149 
 P-Value: 0.558513
Epoch [344/400], Loss: 0.0944
Correlation: -0.042316 
 P-Value: 0.556946
Epoch [345/400], Loss: 0.0944
Correlation: -0.042699 
 P-Value: 0.553380
Epoch [346/400], Loss: 0.0944
Correlation: -0.043158 
 P-Value: 0.549127
Epoch [347/400], Loss: 0.0944
Correlation: -0.043324 
 P-Value: 0.547589
Epoch [348/400], Loss: 0.0944
Correlation: -0.043130 
 P-Value: 0.549389
Epoch [349/400], Loss: 0.0944
Correlation: -0.042618 
 P-Value: 0.554140
Epoch [350/400], Loss: 0.0944
Correlation: -0.041982 
 P-Value: 0.560070
Epoch [351/400], Loss: 0.0944
Correlation: -0.04167

In [233]:
traning_data_6 = torch.tensor(snp_yeast_matrix[top2000_index_6].values, dtype = torch.float)
answer_6 = torch.tensor(YPGLYCEROL, dtype = torch.float)
train_set_6 = dt.TensorDataset(traning_data_6, answer_6)

train_dataset_6, test_dataset_6 = dt.random_split(train_set_6, [train_size, test_size])
train_loader_6 = dt.DataLoader(dataset=train_dataset_6, batch_size=batch_size)
test_loader_6 = dt.DataLoader(dataset=test_dataset_6, batch_size=batch_size)

net_6 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_6.train_net(num_epochs, train_loader_6, test_loader_6)

Correlation: 0.277037 
 P-Value: 0.000088
Epoch [1/400], Loss: 0.2305
Correlation: 0.252326 
 P-Value: 0.000373
Epoch [2/400], Loss: 0.1062
Correlation: 0.245611 
 P-Value: 0.000538
Epoch [3/400], Loss: 0.0722
Correlation: 0.148924 
 P-Value: 0.037723
Epoch [4/400], Loss: 0.0546
Correlation: 0.073624 
 P-Value: 0.306366
Epoch [5/400], Loss: 0.0487
Correlation: 0.124698 
 P-Value: 0.082405
Epoch [6/400], Loss: 0.0465
Correlation: 0.211118 
 P-Value: 0.003050
Epoch [7/400], Loss: 0.0416
Correlation: 0.169238 
 P-Value: 0.018022
Epoch [8/400], Loss: 0.0398
Correlation: 0.065115 
 P-Value: 0.365781
Epoch [9/400], Loss: 0.0377
Correlation: 0.018593 
 P-Value: 0.796412
Epoch [10/400], Loss: 0.0367
Correlation: -0.054009 
 P-Value: 0.453317
Epoch [11/400], Loss: 0.0361
Correlation: -0.130423 
 P-Value: 0.069169
Epoch [12/400], Loss: 0.0356
Correlation: -0.126219 
 P-Value: 0.078702
Epoch [13/400], Loss: 0.0354
Correlation: -0.172135 
 P-Value: 0.016117
Epoch [14/400], Loss: 0.0351
Correlation

Epoch [114/400], Loss: 0.0328
Correlation: -0.064062 
 P-Value: 0.373608
Epoch [115/400], Loss: 0.0328
Correlation: -0.064145 
 P-Value: 0.372990
Epoch [116/400], Loss: 0.0328
Correlation: -0.064140 
 P-Value: 0.373021
Epoch [117/400], Loss: 0.0328
Correlation: -0.064208 
 P-Value: 0.372515
Epoch [118/400], Loss: 0.0328
Correlation: -0.064205 
 P-Value: 0.372539
Epoch [119/400], Loss: 0.0328
Correlation: -0.064147 
 P-Value: 0.372972
Epoch [120/400], Loss: 0.0328
Correlation: -0.064321 
 P-Value: 0.371671
Epoch [121/400], Loss: 0.0328
Correlation: -0.063611 
 P-Value: 0.376985
Epoch [122/400], Loss: 0.0328
Correlation: -0.063645 
 P-Value: 0.376728
Epoch [123/400], Loss: 0.0328
Correlation: -0.064059 
 P-Value: 0.373625
Epoch [124/400], Loss: 0.0328
Correlation: -0.063521 
 P-Value: 0.377658
Epoch [125/400], Loss: 0.0328
Correlation: -0.063730 
 P-Value: 0.376088
Epoch [126/400], Loss: 0.0328
Correlation: -0.063727 
 P-Value: 0.376113
Epoch [127/400], Loss: 0.0328
Correlation: -0.06324

Epoch [226/400], Loss: 0.0328
Correlation: -0.056731 
 P-Value: 0.430844
Epoch [227/400], Loss: 0.0328
Correlation: -0.056400 
 P-Value: 0.433542
Epoch [228/400], Loss: 0.0328
Correlation: -0.057474 
 P-Value: 0.424822
Epoch [229/400], Loss: 0.0328
Correlation: -0.058336 
 P-Value: 0.417897
Epoch [230/400], Loss: 0.0328
Correlation: -0.058033 
 P-Value: 0.420329
Epoch [231/400], Loss: 0.0328
Correlation: -0.056764 
 P-Value: 0.430575
Epoch [232/400], Loss: 0.0328
Correlation: -0.056088 
 P-Value: 0.436099
Epoch [233/400], Loss: 0.0328
Correlation: -0.056670 
 P-Value: 0.431341
Epoch [234/400], Loss: 0.0328
Correlation: -0.057095 
 P-Value: 0.427893
Epoch [235/400], Loss: 0.0328
Correlation: -0.057786 
 P-Value: 0.422305
Epoch [236/400], Loss: 0.0328
Correlation: -0.058364 
 P-Value: 0.417676
Epoch [237/400], Loss: 0.0328
Correlation: -0.058010 
 P-Value: 0.420515
Epoch [238/400], Loss: 0.0328
Correlation: -0.057298 
 P-Value: 0.426248
Epoch [239/400], Loss: 0.0328
Correlation: -0.05633

Epoch [338/400], Loss: 0.0328
Correlation: -0.063927 
 P-Value: 0.374615
Epoch [339/400], Loss: 0.0328
Correlation: -0.050129 
 P-Value: 0.486464
Epoch [340/400], Loss: 0.0328
Correlation: -0.047951 
 P-Value: 0.505622
Epoch [341/400], Loss: 0.0328
Correlation: -0.056516 
 P-Value: 0.432598
Epoch [342/400], Loss: 0.0328
Correlation: -0.066509 
 P-Value: 0.355591
Epoch [343/400], Loss: 0.0328
Correlation: -0.067461 
 P-Value: 0.348731
Epoch [344/400], Loss: 0.0328
Correlation: -0.058257 
 P-Value: 0.418535
Epoch [345/400], Loss: 0.0328
Correlation: -0.051458 
 P-Value: 0.474963
Epoch [346/400], Loss: 0.0328
Correlation: -0.052236 
 P-Value: 0.468305
Epoch [347/400], Loss: 0.0328
Correlation: -0.057584 
 P-Value: 0.423936
Epoch [348/400], Loss: 0.0328
Correlation: -0.061539 
 P-Value: 0.392752
Epoch [349/400], Loss: 0.0328
Correlation: -0.061247 
 P-Value: 0.395012
Epoch [350/400], Loss: 0.0328
Correlation: -0.057726 
 P-Value: 0.422795
Epoch [351/400], Loss: 0.0328
Correlation: -0.05316

In [234]:
traning_data_7 = torch.tensor(snp_yeast_matrix[top2000_index_7].values, dtype = torch.float)
answer_7 = torch.tensor(YPRIBOSE, dtype = torch.float)
train_set_7 = dt.TensorDataset(traning_data_7, answer_7)

train_dataset_7, test_dataset_7 = dt.random_split(train_set_7, [train_size, test_size])
train_loader_7 = dt.DataLoader(dataset=train_dataset_7, batch_size=batch_size)
test_loader_7 = dt.DataLoader(dataset=test_dataset_7, batch_size=batch_size)

net_7 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_7.train_net(num_epochs, train_loader_7, test_loader_7)

Correlation: 0.219602 
 P-Value: 0.002038
Epoch [1/400], Loss: 0.1681
Correlation: 0.205421 
 P-Value: 0.003966
Epoch [2/400], Loss: 0.0738
Correlation: 0.226071 
 P-Value: 0.001484
Epoch [3/400], Loss: 0.0455
Correlation: 0.183800 
 P-Value: 0.010109
Epoch [4/400], Loss: 0.0326
Correlation: 0.114166 
 P-Value: 0.112017
Epoch [5/400], Loss: 0.0285
Correlation: 0.137937 
 P-Value: 0.054481
Epoch [6/400], Loss: 0.0281
Correlation: 0.203243 
 P-Value: 0.004376
Epoch [7/400], Loss: 0.0246
Correlation: 0.203708 
 P-Value: 0.004285
Epoch [8/400], Loss: 0.0236
Correlation: 0.163635 
 P-Value: 0.022267
Epoch [9/400], Loss: 0.0221
Correlation: 0.129713 
 P-Value: 0.070710
Epoch [10/400], Loss: 0.0210
Correlation: 0.089947 
 P-Value: 0.211115
Epoch [11/400], Loss: 0.0203
Correlation: 0.036354 
 P-Value: 0.613861
Epoch [12/400], Loss: 0.0200
Correlation: 0.052097 
 P-Value: 0.469495
Epoch [13/400], Loss: 0.0196
Correlation: 0.019567 
 P-Value: 0.786008
Epoch [14/400], Loss: 0.0195
Correlation: 0.

 P-Value: 0.107008
Epoch [116/400], Loss: 0.0179
Correlation: 0.115772 
 P-Value: 0.107027
Epoch [117/400], Loss: 0.0179
Correlation: 0.115848 
 P-Value: 0.106797
Epoch [118/400], Loss: 0.0179
Correlation: 0.115951 
 P-Value: 0.106483
Epoch [119/400], Loss: 0.0179
Correlation: 0.115820 
 P-Value: 0.106880
Epoch [120/400], Loss: 0.0179
Correlation: 0.115665 
 P-Value: 0.107355
Epoch [121/400], Loss: 0.0179
Correlation: 0.115957 
 P-Value: 0.106466
Epoch [122/400], Loss: 0.0179
Correlation: 0.115951 
 P-Value: 0.106481
Epoch [123/400], Loss: 0.0179
Correlation: 0.115813 
 P-Value: 0.106903
Epoch [124/400], Loss: 0.0179
Correlation: 0.115457 
 P-Value: 0.107991
Epoch [125/400], Loss: 0.0179
Correlation: 0.115851 
 P-Value: 0.106786
Epoch [126/400], Loss: 0.0179
Correlation: 0.115770 
 P-Value: 0.107035
Epoch [127/400], Loss: 0.0179
Correlation: 0.116112 
 P-Value: 0.105995
Epoch [128/400], Loss: 0.0179
Correlation: 0.115964 
 P-Value: 0.106442
Epoch [129/400], Loss: 0.0179
Correlation: 0.

 P-Value: 0.123933
Epoch [230/400], Loss: 0.0179
Correlation: 0.110231 
 P-Value: 0.125010
Epoch [231/400], Loss: 0.0179
Correlation: 0.109841 
 P-Value: 0.126361
Epoch [232/400], Loss: 0.0179
Correlation: 0.109805 
 P-Value: 0.126486
Epoch [233/400], Loss: 0.0179
Correlation: 0.109877 
 P-Value: 0.126234
Epoch [234/400], Loss: 0.0179
Correlation: 0.109629 
 P-Value: 0.127097
Epoch [235/400], Loss: 0.0179
Correlation: 0.110058 
 P-Value: 0.125610
Epoch [236/400], Loss: 0.0179
Correlation: 0.110121 
 P-Value: 0.125392
Epoch [237/400], Loss: 0.0179
Correlation: 0.110020 
 P-Value: 0.125742
Epoch [238/400], Loss: 0.0179
Correlation: 0.109304 
 P-Value: 0.128235
Epoch [239/400], Loss: 0.0179
Correlation: 0.109156 
 P-Value: 0.128756
Epoch [240/400], Loss: 0.0179
Correlation: 0.109030 
 P-Value: 0.129200
Epoch [241/400], Loss: 0.0179
Correlation: 0.109262 
 P-Value: 0.128383
Epoch [242/400], Loss: 0.0179
Correlation: 0.109581 
 P-Value: 0.127266
Epoch [243/400], Loss: 0.0179
Correlation: 0.

 P-Value: 0.151219
Epoch [344/400], Loss: 0.0179
Correlation: 0.108849 
 P-Value: 0.129843
Epoch [345/400], Loss: 0.0179
Correlation: 0.112327 
 P-Value: 0.117951
Epoch [346/400], Loss: 0.0179
Correlation: 0.110926 
 P-Value: 0.122635
Epoch [347/400], Loss: 0.0179
Correlation: 0.105095 
 P-Value: 0.143690
Epoch [348/400], Loss: 0.0179
Correlation: 0.097922 
 P-Value: 0.173230
Epoch [349/400], Loss: 0.0179
Correlation: 0.092904 
 P-Value: 0.196433
Epoch [350/400], Loss: 0.0179
Correlation: 0.094424 
 P-Value: 0.189177
Epoch [351/400], Loss: 0.0179
Correlation: 0.103598 
 P-Value: 0.149517
Epoch [352/400], Loss: 0.0179
Correlation: 0.116217 
 P-Value: 0.105676
Epoch [353/400], Loss: 0.0179
Correlation: 0.123806 
 P-Value: 0.084639
Epoch [354/400], Loss: 0.0179
Correlation: 0.116318 
 P-Value: 0.105372
Epoch [355/400], Loss: 0.0179
Correlation: 0.098959 
 P-Value: 0.168705
Epoch [356/400], Loss: 0.0179
Correlation: 0.084076 
 P-Value: 0.242572
Epoch [357/400], Loss: 0.0179
Correlation: 0.

In [235]:
traning_data_8 = torch.tensor(snp_yeast_matrix[top2000_index_8].values, dtype = torch.float)
answer_8 = torch.tensor(YPSORBITOL, dtype = torch.float)
train_set_8 = dt.TensorDataset(traning_data_8, answer_8)

train_dataset_8, test_dataset_8 = dt.random_split(train_set_8, [train_size, test_size])
train_loader_8 = dt.DataLoader(dataset=train_dataset_8, batch_size=batch_size)
test_loader_8 = dt.DataLoader(dataset=test_dataset_8, batch_size=batch_size)

net_8 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_8.train_net(num_epochs, train_loader_8, test_loader_8)

Correlation: 0.048163 
 P-Value: 0.503739
Epoch [1/400], Loss: 0.1999
Correlation: 0.056231 
 P-Value: 0.434926
Epoch [2/400], Loss: 0.1089
Correlation: 0.044361 
 P-Value: 0.538033
Epoch [3/400], Loss: 0.0626
Correlation: -0.015074 
 P-Value: 0.834310
Epoch [4/400], Loss: 0.0456
Correlation: -0.064635 
 P-Value: 0.369337
Epoch [5/400], Loss: 0.0368
Correlation: -0.072988 
 P-Value: 0.310570
Epoch [6/400], Loss: 0.0360
Correlation: -0.042550 
 P-Value: 0.554766
Epoch [7/400], Loss: 0.0324
Correlation: -0.011028 
 P-Value: 0.878400
Epoch [8/400], Loss: 0.0296
Correlation: -0.038738 
 P-Value: 0.590808
Epoch [9/400], Loss: 0.0281
Correlation: -0.064426 
 P-Value: 0.370889
Epoch [10/400], Loss: 0.0267
Correlation: -0.058651 
 P-Value: 0.415388
Epoch [11/400], Loss: 0.0259
Correlation: -0.060218 
 P-Value: 0.403012
Epoch [12/400], Loss: 0.0256
Correlation: -0.045035 
 P-Value: 0.531869
Epoch [13/400], Loss: 0.0253
Correlation: -0.045391 
 P-Value: 0.528628
Epoch [14/400], Loss: 0.0251
Corr

Epoch [115/400], Loss: 0.0230
Correlation: 0.063032 
 P-Value: 0.381355
Epoch [116/400], Loss: 0.0230
Correlation: 0.062807 
 P-Value: 0.383056
Epoch [117/400], Loss: 0.0230
Correlation: 0.063223 
 P-Value: 0.379913
Epoch [118/400], Loss: 0.0230
Correlation: 0.063236 
 P-Value: 0.379809
Epoch [119/400], Loss: 0.0230
Correlation: 0.062963 
 P-Value: 0.381878
Epoch [120/400], Loss: 0.0230
Correlation: 0.062966 
 P-Value: 0.381853
Epoch [121/400], Loss: 0.0230
Correlation: 0.063113 
 P-Value: 0.380736
Epoch [122/400], Loss: 0.0230
Correlation: 0.063193 
 P-Value: 0.380133
Epoch [123/400], Loss: 0.0230
Correlation: 0.063142 
 P-Value: 0.380522
Epoch [124/400], Loss: 0.0230
Correlation: 0.062798 
 P-Value: 0.383121
Epoch [125/400], Loss: 0.0230
Correlation: 0.063160 
 P-Value: 0.380379
Epoch [126/400], Loss: 0.0230
Correlation: 0.063239 
 P-Value: 0.379792
Epoch [127/400], Loss: 0.0230
Correlation: 0.063029 
 P-Value: 0.381376
Epoch [128/400], Loss: 0.0230
Correlation: 0.062927 
 P-Value: 0

 P-Value: 0.360410
Epoch [229/400], Loss: 0.0230
Correlation: 0.065615 
 P-Value: 0.362110
Epoch [230/400], Loss: 0.0230
Correlation: 0.065593 
 P-Value: 0.362266
Epoch [231/400], Loss: 0.0230
Correlation: 0.065826 
 P-Value: 0.360562
Epoch [232/400], Loss: 0.0230
Correlation: 0.065750 
 P-Value: 0.361116
Epoch [233/400], Loss: 0.0230
Correlation: 0.065825 
 P-Value: 0.360569
Epoch [234/400], Loss: 0.0230
Correlation: 0.065941 
 P-Value: 0.359726
Epoch [235/400], Loss: 0.0230
Correlation: 0.066068 
 P-Value: 0.358797
Epoch [236/400], Loss: 0.0230
Correlation: 0.066284 
 P-Value: 0.357230
Epoch [237/400], Loss: 0.0230
Correlation: 0.065997 
 P-Value: 0.359314
Epoch [238/400], Loss: 0.0230
Correlation: 0.066012 
 P-Value: 0.359199
Epoch [239/400], Loss: 0.0230
Correlation: 0.066133 
 P-Value: 0.358324
Epoch [240/400], Loss: 0.0230
Correlation: 0.066087 
 P-Value: 0.358659
Epoch [241/400], Loss: 0.0230
Correlation: 0.066179 
 P-Value: 0.357989
Epoch [242/400], Loss: 0.0230
Correlation: 0.

 P-Value: 0.363616
Epoch [343/400], Loss: 0.0230
Correlation: 0.065462 
 P-Value: 0.363231
Epoch [344/400], Loss: 0.0230
Correlation: 0.065626 
 P-Value: 0.362030
Epoch [345/400], Loss: 0.0230
Correlation: 0.066145 
 P-Value: 0.358235
Epoch [346/400], Loss: 0.0230
Correlation: 0.066324 
 P-Value: 0.356929
Epoch [347/400], Loss: 0.0230
Correlation: 0.066150 
 P-Value: 0.358196
Epoch [348/400], Loss: 0.0230
Correlation: 0.065830 
 P-Value: 0.360536
Epoch [349/400], Loss: 0.0230
Correlation: 0.066095 
 P-Value: 0.358600
Epoch [350/400], Loss: 0.0230
Correlation: 0.066737 
 P-Value: 0.353936
Epoch [351/400], Loss: 0.0230
Correlation: 0.066787 
 P-Value: 0.353575
Epoch [352/400], Loss: 0.0230
Correlation: 0.066453 
 P-Value: 0.355997
Epoch [353/400], Loss: 0.0230
Correlation: 0.066264 
 P-Value: 0.357371
Epoch [354/400], Loss: 0.0230
Correlation: 0.066298 
 P-Value: 0.357122
Epoch [355/400], Loss: 0.0230
Correlation: 0.066694 
 P-Value: 0.354253
Epoch [356/400], Loss: 0.0230
Correlation: 0.